# What is MDAnalysis ?


**MDAnalysis** is a Python open-source toolkit dedicated to the analysis of molecular dynamics trajectories. It is developed and maintained by a global community of scientists and it is free.  
It provides powerful and flexible framework for working with MD data, allowing to analyze trajectory files from various simulation packages.  
It was initially presented in a [paper released in 2011](https://doi.org/10.1002/jcc.21787) and a [second paper was published in 2016](https://doi.org/10.25080/majora-629e541a-00e) to describe many updates in 2016 for a better global description of the toolkit. The current version is the 2.6.1.



# Installation

Our custom conda environment:
We provide you a yaml file that you can use to generate a conda environment containing all the necessary packages to work with MDAnalysis.

Just run this command in your terminal :
conda env create -f mdanalysis_conda_env.yml

pip:
You can otherwise run the following command to install it with core dependencies :
pip install --upgrade MDAnalysis

# Universe

The Main data structures in the User Interface are Universe and AtomGroup. With them we could do selections, units, topology I/O, trajectory I/O… 

Universe is a class in which the data is loaded. It is the primary object and core interface to Molecular Dynamics and used to Read topology information from a file.

To create a Universe object, we need a topology file such as a PDB file that contains all static information like atoms name and mass, charges, bonds and connectivity… A trajectory file is optionally used in the creation of the Universe object but it must have the coordinates in the same order as atoms in the topology file. 
Universe has two key properties: 
	1. atoms: Systems' atoms
	2. trajectory: the loaded trajectory reader

There are several formats of Universe objects that one can load: 
u=Universe(topology,trajectory)
u=Universe(pdbfile)                                                    
u=Universe(topology,[traj1,traj2,...])                        
u=Universe(topology,traj1,traj2,...)    

In [ ]:
import MDAnalysis as mda

# Creation of a Universe using a topology and a trajectory file
u = mda.Universe('PDB_tubuline.pdb', 'trajectoire_tubuline.dcd')

# Creating a universe using only the coordinates from the second file only
u1 = mda.Universe('PDB_tubuline.pdb', 'trajectoire_tubuline.dcd', all_coordinates = False)
print(u1.trajectory)

# Creating a universe using only the coordinates from both files
u2 = mda.Universe('PDB_tubuline.pdb', 'trajectoire_tubuline.dcd', all_coordinates = True)
print(u2.trajectory)

In [ ]:
# A universe can also be created from scratch
# In this examplewe will create a universe of 1000 water molecules
# For this example we will need the numpy
import numpy as np
n_residues = 1000 # Assigning the number of water molecules
n_atoms = n_residues * 3 # Because a Water molecule contains 3 atoms

# Creating a list of indicies for the residues
resindicies = np.repeat(range(n_residues), 3) 
assert len(resindicies) == n_atoms

# Assigning the water molecules to 1 segment
segindicies =[0] * n_residues

# Creating the universe using
sol = mda.Universe.empty(n_atoms, n_residues = n_residues, atom_resindex = resindicies, residue_segindex = segindicies)
sol

# Selection

In [ ]:
# AtomGroups are constructed by atoms
atom1 = u.atoms[4]
atom2 = u.atoms[5]
ag = mda.AtomGroup([atom1,atom2])
print(ag)

In [ ]:
# AtomGroups are also created by manipulating another AtomGroup
# Creating an AtomGroup of atoms present in ARG residues
ag = u.atoms.select_atoms('resname ARG')
#print(ag)

# Atomgroups can be indexed, and here we are printing the first atom of the Universe
print(u.atoms[0])

# It is also possible to slice them, here we are printing atoms at index 0, 5 and 10
ag2 = u.atoms[0:11:5] 
print(ag2)

# Splicing returns another AtomGroup, we can still check their indicies in the primary AtomGroup.
ag2.indices

# You can use group and set operators on atom groups
group1 = u.atoms[1:5]
group2 = u.atoms[10:3:-1]

# Concatinating AtomGroups will preserve their order and the duplicates 
grouping = group1 + group2
grouping.indices

In [ ]:
# However union will create an AtomGroup with unique atoms ordered according to their topology index
union = group1 | group2
union.indices

In [ ]:
# You can split or group AtomGroups by their level of connectivity such as residue, molecule, or segment
group3 = u.atoms[:100]
group3.split('residue')

In [ ]:
# Or their topology attributes such as masses
group3.groupby('masses')

In [ ]:
# Selection of atoms
# You can select atoms using selection keywords
# They are case sensitive, sequentially ordered, and selection are parsed from left to right with parathesis used for grouping
# String selections for names and resdiues are matched with wildcards
# * for any number of any character
# ? for any single character
# [seq] matches the sequence
# [!seq] matches any char not in seq
# [!?] selects empty string

u.select_atoms("protein") # Selects the entire protein

In [ ]:
u.select_atoms("backbone") # Selects the backbone atoms

In [ ]:
u.select_atoms("segid PROA") # Selects by segment ID as given in the topology file

In [ ]:
u.select_atoms("resnum 1:5") # Selects every atom that are in the residies 1 to 5

In [ ]:
u.select_atoms("resname ALA") # Selection by the 3 letter code of the resname

In [ ]:
u.select_atoms("name CA") # Selection by name of atom, here selection of C alpha atoms

In [ ]:
# Selection types can be combined
print(u.select_atoms("segid PROA and resnum 1 and name SD")) # Selection of the SD atom from the PROA segment of the first residue
print(u.select_atoms("segid PROA and resnum 1 and not name H"))# Selection of the all atoms excpet H from the PROA segment of the first residue

# Storage and visualization tools

# Data storage
The starting point when working with molecular dynamics is to know the file you
are using and their extension. You can refer to this [official list](https://userguide.mdanalysis.org/stable/formats/index.html) to see the
the format supported by MDAnalysis.
Data can be stored in multiple raw forms such as a data file or transformed
forms like a plot or a movie, some of them will be presented in this section.



## Writing out data

### Frame and trajectories
MDAnalysis allows you to write frames or trajectories in any of the supported format with `.write()`:

In [ ]:
ag = u.select_atoms("name CA")

# Writing the current frame to a file
ag.write("c-alpha.pdb")

# Writing the trajectory using the 'frames' keyword
ag.write('c-alpha_all.xtc', frames='all')

# Example for slice or index
ag.write('c-alpha_skip2.trr', frames=u.trajectory[::2])
ag.write('c-alpha_some.dcd', frames=u.trajectory[[0,2,3]])

There is also the possiblity to write by iterating through the frames of a trajectory with `Writer()`:

In [ ]:
with mda.Writer('c-alpha.xyz', ag.n_atoms) as w:
    for ts in u.trajectory:
        w.write(ag)

### Selection of atoms
MDAnalysis also allows you to write out a selection of atoms to a format readable by [other programs](https://userguide.mdanalysis.org/stable/formats/selection_exporters.html#id2). For that, the `AtomGroup.write` method can be used with the appropriate file extension:

In [ ]:
ag = u.select_atoms('resname ALA')

# Exporting the alanines as a vmd representation
ag.write("ala_selection.vmd", name="alanine")

Additionnal keywords can be taken such as `mode` which allows to alter the writing behavior:

In [ ]:
# Use mode='a' to append
u.select_atoms('resname PRO').write('residues.ndx',
                                    name='PRO',
                                    mode='w')
# Use mode='w' to overwrite
u.select_atoms('resname GLY').write('residues.ndx',
                                    name='GLY',
                                    mode='a')

# Visualisation

## Plotting trajectories
Generally, trajectories are worked on through iteration so that only the
information of the active frame are considered. A way to visualize specific
information from each frame is to extract them in a list and convert them into
other data structures such as a *Numpy* array or a *Pandas* DataFrame. This way,
plotting with *Matplotlib* can be easier to do:

### Radius of gyration using *Pandas*

In [ ]:
rgyr = []
time = []
protein = u.select_atoms("protein")
for ts in u.trajectory:
    time.append(u.trajectory.time)
    rgyr.append(protein.radius_of_gyration())

In [ ]:
import pandas as pd
rgyr_df = pd.DataFrame(rgyr, columns=['Radius of gyration (A)'], index=time)
rgyr_df.index.name = 'Time (ps)'

rgyr_df.head()

In [ ]:
rgyr_df.plot(title='Radius of gyration')                        
plt.show()

### All distances between two selections using *Numpy*

In [ ]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PDB_small
from MDAnalysis.analysis import distances

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
# suppress some MDAnalysis warnings when writing PDB files
warnings.filterwarnings('ignore')

In [ ]:
u = mda.Universe(PDB_small)   # open AdK

In [ ]:
LID_ca = u.select_atoms('name CA and resid 122-159')
NMP_ca = u.select_atoms('name CA and resid 30-59')

n_LID = len(LID_ca)
n_NMP = len(NMP_ca)
print('LID has {} residues and NMP has {} residues'.format(n_LID, n_NMP))

In [ ]:
dist_arr = distances.distance_array(LID_ca.positions, # reference
                                    NMP_ca.positions, # configuration
                                    box=u.dimensions)
dist_arr.shape

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(dist_arr, origin='upper')

# add residue ID labels to axes
tick_interval = 5
ax.set_yticks(np.arange(n_LID)[::tick_interval])
ax.set_xticks(np.arange(n_NMP)[::tick_interval])
ax.set_yticklabels(LID_ca.resids[::tick_interval])
ax.set_xticklabels(NMP_ca.resids[::tick_interval])

# add figure labels and titles
plt.ylabel('LID')
plt.xlabel('NMP')
plt.title('Distance between alpha-carbon')

# colorbar
cbar = fig.colorbar(im)
cbar.ax.set_ylabel('Distance (Angstrom)')

## NGLView


[NGL Viewer](http://nglviewer.org/) is an Interactive widget to visualize molecular structures or trajectories in Jupyter notebooks.

You can see [this tutorial](https://projects.volkamerlab.org/teachopencadd/talktorials/T017_advanced_nglview_usage.html#Custom-coloring-schemes-and-representations) from the Volkamerlab for a walkthrough of the NGLView usage.

#### Visualize the starting structure
We first use `.show_file()` to visualize the pdb structure of the tubulin.

In [ ]:
import nglview as nv


# load the tubuline PDB file and display viewer widget
pdb_view = nv.show_file("PDB_tubuline.pdb")
pdb_view.center()

pdb_view

#### Visualize your mdanalysis object
For the visualization of a mdanalysis object you can use `show_mdanalysis(univ)` that shows `MDAnalysis` Universe or AtomGroup `univ`.

In [ ]:
import MDAnalysis as mda


reference = mda.Universe('PDB_tubuline.pdb', 'trajectoire_tubuline.dcd')
univ_view = nv.show_mdanalysis(reference)

# Set the view on the frame 100
univ_view.frame = 100

univ_view

##### Visualizing the first and last structures of the trajectory
It can be interesting to compare frames taken from different conformations adopted during the simulation. Thus we provide an example on how to do this comparison on the first and the last frames of our simulation.  

For a better visualization of the structural differences, a simple alignement will be performed. As the most fluctuating part of the protein where found from the residues 415 to the end, this alignement will be made one the $C\alpha$ of the residues 0 to 415 of the protein.

In [ ]:
from MDAnalysis.analysis import align, rms


# Create an independent copy of the mobile Universe
ref = mobile.copy()

# Set the mobile trajectory on the last frame
mobile.trajectory[-1]

# Perform a structure alignment
rmsds = align.alignto(mobile,  # mobile
                      reference,  # reference
                      select='name CA and resid 0-415',  # selection to operate on
                      match_atoms=True)  # whether to match atoms
print(rmsds)

# Create an ngl view of the structures merged into the same Universe
ex_frames_view = nv.show_mdanalysis(mda.Merge(traj_clone_1.atoms, traj_clone_2.atoms))
ex_frames_view

##### Comparative visualization the most mobile part movements of the tubuline

In [ ]:
# set mobile trajectory to last frame
mobile.trajectory[-1]
# set reference trajectory to first frame
ref.trajectory[0]

# Select the alpha carbons of each universe
mobile_ca = mobile.select_atoms('name CA')
ref_ca = ref.select_atoms('name CA')

# Calculate the RMSD on the alpha carbons
unaligned_rmsd = rms.rmsd(mobile_ca.positions, ref_ca.positions,
                          superposition=False)
print(f"Unaligned RMSD: {unaligned_rmsd:.2f}")

In [ ]:
# Perform a backbone-wise alignment
aligner = align.AlignTraj(mobile, ref, select='backbone',
                          filename='aligned_to_first_frame.dcd').run()

# Provide aligned coordinates for the trajectory
mobile = mda.Universe('PDB_tubuline.pdb', 'aligned_to_first_frame.dcd')

In [ ]:
mobile.trajectory[-1]
ref.trajectory[0]

mobile_ca = mobile.select_atoms('name CA')
ref_ca = ref.select_atoms('name CA')
aligned_rmsd = rms.rmsd(mobile_ca.positions, ref_ca.positions,
                        superposition=False)
print(f"Aligned RMSD: {aligned_rmsd:.2f}")

# Create a view of the mobile trajectory
view = nv.show_mdanalysis(mobile)

# Add the pdb structure in the viewer
view.add_component('PDB_tubuline.pdb')

# Update the cartoon representation of the trajectory 
view.update_cartoon(component=0, color='blue')
view

# Align a trajectory on a reference

In order to obtain the correct RMSD or RMSF value of the structure of the tubuline after the simulation, we need to align the protein at the begining of the simulation to itself but on the last frame of the trajectory.

To be able to do that, we will need to use a component of the anaylis module provided by MDAnalysis.




In [ ]:
from MDAnalysis.analysis import align

The analysis module contains a lot of different algorithms to extract data from the topologie and the trajectory of our system. 

To perform the alignement, we will need a reference structure and a mobile one which will be aligned onto the reference.

In our case we will select the orignal pdb structure as our reference point and the tubuline in the last frame as our mobile structure.

To do that will use the function *alignto* from the align module. This function takes 8 arguments:

required:
- mobile (Universe | AtomGroup): the structure that will be aligned
- reference (Universe | AtomGroup): the structure on which the alignement is based on

optional:
- select (str | dict | tuple): filter on which groups of atoms are chosen for the alignment. If a single string is used, the selection need to be valid for both the mobile and the reference AtomGroups. We can perform seperate selection filters by using a dictionnary (i.e. {'mobile': 'mySelection1', 'reference': 'mySelection2}) or a tuple.
- match_atoms (bool): align the mobile and reference structure atom by atom.
- weights: ("mass" | None | array): used in case you want to do a weighted RMSD alignement or have a different center of mass. You can either use the mass of the atoms or input an array with custom weights.
- tol_mass (float): Threshold where the alignment will be aborted if two matching atoms differ by more than tol_mass
- strict (bool): Raise an error if an atom is different in the mobile or reference selections.

And in returns, we obtain a tuple containing the RMSD before the alignment and the RMSD after.

In [ ]:
reference = mda.Universe('PDB_tubuline.pdb')
mobile = mda.Universe('PDB_tubuline.pdb', 'trajectoire_tubuline.dcd')

# Take the last frame of the simulation
mobile.trajectory[-1]

align.alignto(mobile, reference, select='protein and name CA', weights='mass')


# Determinate flexible loops with RMSF

What means RMSF and what is it use for ?

Before explaining RMSF, let me introduce RMSD. 
- RMSD means Root Mean Square Deviation. It represents the difference between two structures and in the case of simulations, the motion of our whole structure of interest throughout the simulation. It is basically plotted as structure variation vs. time.

- RMSF means Root Mean Square Fluctuation. It also shows fluctuations all over the simulations but for specific atoms or residues. It can highlight which residue or part of our structure are the most flexible. It is also plotted as residue/part of structure vs. time.

Now that we have explained the theory, let's look at a practical example with our tubuline.

In [ ]:
# Import specific package for analysis and in this case : rms (rmsd, rmsf) 
from MDAnalysis.analysis import rms

# First you have to select what part of your structure you want to look at
# For now we take as example the alpha carbons of the tubulin
c_alphas = u.select_atoms('protein and name CA')

# Calculate the rmsf for alpha carbons 
R = rms.RMSF(c_alphas).run()

In [ ]:
# Import package for plotting
import matplotlib.pyplot as plt

# Then plot your RMSF of alpha carbons
# A graph will be displayed with RMSF values in Angstroms on y-axis and the residue indices on x-axis
plt.plot(c_alphas.resids, R.results.rmsf)

# Other types of analysis featured by MDAnalysis

In addition to the functionalities presented previously, MDAnalysis offers other analysis tools.

## Distances and contacts

You can compute distances between atoms, residues or larger sets. It can give you valuable datas :
- How residues are close during the simulation
- If two residues are in contact if they are within a certain distance
- An overview of the global movement of the system by looking at number of contacts all over the simulation

## Structure

- Having an idea of dihedral angles and secondary structures
- Calculating RDF (Radial Function Distribution) of solvent around certain atoms


## Hydrogen bonds

MDAnalysis has features to analyse hydrogen bonds in a Universe thanks to the function `HydrogenBondAnalysis`